In [90]:
import pandas as pd
import json
import os

In [49]:
with open('attack_pose/translation_mappings.json') as f:
    mappings = json.load(f)

# Convert excel files to json

In [97]:
def pose_df_to_dict(pose_df):

	annot = {'annotations': []}

	# Keypoints in json files used by the pose estimation model
	# "keypoints": [
	# 				"nose",
	# 				"left_eye",
	# 				"right_eye",
	# 				"left_ear",
	# 				"right_ear",
	# 				"left_shoulder",
	# 				"right_shoulder",
	# 				"left_elbow",
	# 				"right_elbow",
	# 				"left_wrist",
	# 				"right_wrist",
	# 				"left_hip",
	# 				"right_hip",
	# 				"left_knee",
	# 				"right_knee",
	# 				"left_ankle",
	# 				"right_ankle"
	#                 ]
	
	for i, row in pose_df.iterrows():
		keypoints = [
		row['nose_y'],
		row['nose_x'],
		row['nose_p'],
		
		row['left_eye_y'],
		row['left_eye_x'],
		row['left_eye_p'],
		
		row['right_eye_y'],
		row['right_eye_x'],
		row['right_eye_p'],
		
		row['left_ear_y'],
		row['left_ear_x'],
		row['left_ear_p'],
		
		row['right_ear_y'],
		row['right_ear_x'],
		row['right_ear_p'],
		
		row['left_shoulder_y'],
		row['left_shoulder_x'],
		row['left_shoulder_p'],
		
		row['right_shoulder_y'],
		row['right_shoulder_x'],
		row['right_shoulder_p'],
		
		row['left_elbow_y'],
		row['left_elbow_x'],
		row['left_elbow_p'],
		
		row['right_elbow_y'],
		row['right_elbow_x'],
		row['right_elbow_p'],
		
		row['left_wrist_y'],
		row['left_wrist_x'],
		row['left_wrist_p'],
		
		row['right_wrist_y'],
		row['right_wrist_x'],
		row['right_wrist_p'],
		
		row['left_hip_y'],
		row['left_hip_x'],
		row['left_hip_p'],
		
		row['right_hip_y'],
		row['right_hip_x'],
		row['right_hip_p'],
		
		row['left_knee_y'],
		row['left_knee_x'],
		row['left_knee_p'],
		
		row['right_knee_y'],
		row['right_knee_x'],
		row['right_knee_p'],
		
		row['left_ankle_y'],
		row['left_ankle_x'],
		row['left_ankle_p'],
		
		row['right_ankle_y'],
		row['right_ankle_x'],
		row['right_ankle_p'],
	]
		frame_num = row['frame_number']

		category_id = 1
		bbox = [0 for _ in range(4)]
		score = 1
		area = 0
		b_score = 1
		object_id = 0
		objects = [
			{
				'category_id': category_id,
				'bbox': bbox,
				'score': score,
				'keypoints': keypoints,
				'area': area,
				'b_score': b_score,
				'object_id': object_id,
			}
		]
		annot['annotations'].append(
			{
				'frame_num': frame_num,
				'objects': objects,
			}
		) 
	return annot


def pose_excel_to_json(excel_file_name, json_file_name, translation_mappings):
	df = pd.read_excel(excel_file_name)
	df.columns = [translation_mappings[col] for col in df.columns.values]
	annot = pose_df_to_dict(df)
	with open(json_file_name, 'w') as f:
		json.dump(annot, f, indent=4)

In [98]:
# get all excel files from directory
excel_files = [f for f in os.listdir('attack_pose/excel_files/') if f.endswith('.xlsx')]
for excel_file in excel_files:
    pose_excel_to_json(
        excel_file_name=os.path.join('attack_pose/excel_files/', excel_file),
        json_file_name=os.path.join('attack_pose/json_files/', excel_file.replace('.xlsx', '.json')),
        translation_mappings=mappings,
    )


# Check if all correct

In [85]:
def json2annotations(json_dir):
    with open(json_dir) as f:
        jointDict = json.load(f)
    # case for cv-api directly using
    return jointDict['annotations'] if 'annotations' in jointDict else jointDict['result']['annotations']

In [111]:
annot_new = json2annotations('attack_pose/json_files/GX010032_Clip_10_sec_24_POSE_tabel.json')
for key in annot_new[0]['objects'][0].keys():
    if key != 'keypoints':
        print(key, annot_new[0]['objects'][0][key])
    else:
        print(key, len(annot_new[0]['objects'][0][key]))

category_id 1
bbox [0, 0, 0, 0]
score 1
keypoints 51
area 0
b_score 1
object_id 0


In [112]:
annot = json2annotations('bpe-datasets/refined_skeleton/007/S001C001P004R001A007.json')
for key in annot[0]['objects'][0].keys():
    if key != 'keypoints':
        print(key, annot[0]['objects'][0][key])
    else:
        print(key, len(annot[0]['objects'][0][key]))

category_id 1
bbox [991.48, 231.33, 162.58, 528.06]
score 0.877
keypoints 51
area 19294.643
b_score 0.968
object_id 0
